# Using Bokeh to Create an Interactive Map

This is the second part of the NLP-Example.py code, where the coordinates of the place names within the poem are mapped.

In [7]:
'''
    -*- coding: utf-8 -*-
    Python Version: 3.6
    Course: GEOG5790M; Advanced Skills; @UniversityofLeeds
    Practical: 8; NLTK
    Author: kmbutterfield
    File name: NLP-Example.py
    Description: Finding location names within T.S. Eliot's "The Waste Land".
'''

import os
import pandas as pd
import bokeh
from bokeh.io import output_notebook, show
from pyproj import Proj, transform 
from bokeh.plotting import figure
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.models import (GMapPlot, GMapOptions, ColumnDataSource, 
                          Circle, Range1d, PanTool, ZoomInTool,
                          ZoomOutTool, HoverTool)

''' Reading the CSV file and Assigning Tools '''
# Open the CSV containing locations names.
places = pd.read_csv("places-locations.csv", encoding="cp1252")

# What to show when a dot is hovered over.
hover = HoverTool(tooltips=[
    ("Address", "@address"), ("(Latitude, Longitude)", "(@lat, @long)")])

pan = PanTool()
zoomin = ZoomInTool()
zoomout = ZoomOutTool()

# Show the whole world 
bound = 20000000 # meters

''' Designing the Figure '''
# Creating Figure hover and title
fig = figure(tools=[hover, pan, zoomin, zoomout], x_range=(-bound, bound), y_range=(-bound, bound))
fig.axis.visible = False
fig.add_tile(CARTODBPOSITRON)
fig.title.text = "Names of Locations found in T.S.Eliots 'The Waste Land'"

# Swiching Spatial Reference
inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:3857')

list_lng,list_lat  = [],[]
for index, place in places.iterrows():
    
    longitude, latitude = place.lng, place.lat
    longitude1,latitude1 = transform(inProj,outProj,longitude,latitude)
    
    list_lng.append(longitude1)
    list_lat.append(latitude1)

# Identifying columns for the hover text display
a = {'address': places.address, 'X': list_lng, 'Y': list_lat, 'lat': places.lat, 'long': places.lng}
source = ColumnDataSource(a)

# Show dots on the map of each location named in the poem
circle = fig.circle(x='X', y='Y', alpha=0.5, color="green", size=10, source=source)

''' Show the igure in a new tab '''
show(fig)